# Evaluating Retrieval-Augmented Generation (RAG) Applications

Evaluating a Retrieval-Augmented Generation (RAG) system requires different strategies depending on whether ground truth data is available.

What Is Ground Truth?

Ground truth refers to validated, correct answers to user queries—often called gold answers.
A collection of queries paired with their gold answers forms a Golden Dataset, which serves as a benchmark for measuring the quality and reliability of a RAG system’s outputs.

Golden datasets are especially valuable when evaluating:

- Answer correctness

- Faithfulness to retrieved context

- Overall system performance over time

RAG Evaluation Test Case Structure

A typical RAG evaluation test case consists of the following components:

1. *User Query:*
The input question or prompt provided by the user.

2. *RAG-Generated Response*:
The answer produced by the RAG pipeline using retrieved documents and a language model.

3. *Baseline Validated Answer (Optional)*:
A ground truth (gold) answer used for direct comparison when available.

Note: When ground truth is unavailable, evaluation relies on reference-free or LLM-assisted metrics.

RAG Evaluation Frameworks

To systematically evaluate RAG systems, one of the following widely adopted frameworks is typically used:

1. RAG Triad

The RAG Triad evaluates a RAG pipeline across three core dimensions:

- *Context Relevance*:
How relevant the retrieved documents are to the user query.

- *Answer Faithfulness (Groundedness)*:
Whether the generated answer is supported by the retrieved context.

- *Answer Relevance*:
How well the generated answer addresses the user query.

This framework is particularly useful when ground truth answers are unavailable.


2. RAGAS (Retrieval-Augmented Generation Assessment)

RAGAS is a more comprehensive evaluation framework designed specifically for RAG systems. It provides a set of metrics that assess both retrieval and generation quality, such as:

- Context precision and recall

- Answer relevance

- Faithfulness

- Answer correctness (when ground truth is available)

RAGAS supports both ground-truth-based and reference-free evaluation, making it suitable for real-world production scenarios.

In [1]:
# Installing the required libraries
!pip install -q openai==1.66.3 \
                tiktoken==0.9.0 \
                pypdf==5.4.0 \
                langchain==0.3.20 \
                langchain-community==0.3.19 \
                langchain-chroma==0.2.2 \
                langchain-openai==0.3.9 \
                chromadb==0.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.1/602.1 kB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458

**Importing the Libraries**


In [3]:
# Importing the standard Libraries
import time
from datetime import datetime

# ChromaDB Vector Database
import chromadb

# OpenAI SDK
from openai import OpenAI

# LangChain Utilities
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Loads all PDF files from a directory and extracts text from each.
from langchain_community.document_loaders import PyPDFDirectoryLoader

# Base class representing a document in LangChain; useful for downstream chaining and processing.
from langchain_core.documents import Document

# Embeddings and Vector Store
from langchain_openai import OpenAIEmbeddings

# Integration for using Chroma as the vector store within LangChain’s ecosystem
from langchain_chroma import Chroma

#ignore all warings
import warnings
warnings.filterwarnings('ignore')

#hide warnings from chroma
import logging
logging.getLogger("chromadb").setLevel(logging.CRITICAL)

In [4]:
# Set up the OpenAI API Key
import os
from google.colab import userdata

openai_api_key = userdata.get('OPENAI_API_KEY')
base_url = 'https://aibe.mygreatlearning.com/openai/v1'

client = OpenAI(
    api_key = openai_api_key,
    base_url = base_url
)

model_name ='gpt-4o-mini'

In [5]:
# Unzip the dataset containing the policy document
!unzip PowerBI.zip

Archive:  PowerBI.zip
  inflating: Introducing_Power_BI.pdf  


In [6]:
# Set the directory where PDF files to be stored
pdf_file_loc = '/content/Introducing_Power_BI.pdf'

#load pdf file
loader = PyPDFDirectoryLoader('/content/')

#split document into chunks
def doc_spliter_(document, chunk_size, chunk_overlap):

  spliter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        encoding_name = 'cl100k_base',
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap
    )

  #load and split documents
  chunks = document.load_and_split(spliter)

  if chunks:

    for i, chunk in enumerate(chunks[:5]):
      print(f'\n --- chunk {i+1} --')
      print(f'producer:{chunk.metadata['producer']}')
      print(f'creator:{chunk.metadata['creator']}')
      print(f'creationdate:{chunk.metadata['creationdate']}')
      print(f'author:{chunk.metadata['author']}')
      print(f'title:{chunk.metadata['title']}')
      print(f'Source:{chunk.metadata['source']}')
      print(f'total_pages:{chunk.metadata['total_pages']}')
      print(f'page:{chunk.metadata['page']}')
      print(f'page_label:{chunk.metadata['page_label']}')
      print(f'Length: {len(chunk.page_content)} characters')
      print(f'content:')
      print(chunk.page_content)
      print('-'*40)

  return chunks

# Chunks are stored within LangChain's Document class
chunked_docs = doc_spliter_(loader, 512, 16)


 --- chunk 1 --
producer:Adobe Acrobat Pro 10.1.16
creator:Adobe Acrobat Pro 10.1.16
creationdate:2016-06-13T10:18:21-04:00
author:Joan
title:
Source:/content/Introducing_Power_BI.pdf
total_pages:407
page:0
page_label:1
Length: 63 characters
content:
Introducing
Microsoft 
Power BI
Alberto Ferrari and Marco Russo
----------------------------------------

 --- chunk 2 --
producer:Adobe Acrobat Pro 10.1.16
creator:Adobe Acrobat Pro 10.1.16
creationdate:2016-06-13T10:18:21-04:00
author:Joan
title:
Source:/content/Introducing_Power_BI.pdf
total_pages:407
page:1
page_label:2
Length: 934 characters
content:
PUBLISHED BY 
Microsoft Press 
A division of Microsoft Corporation 
One Microsoft Way 
Redmond, Washington 98052-6399 
Copyright © 2016 by Microsoft Corporation 
All rights reserved. No part of the contents of 
this book may be reproduced or transmitted in 
any form or by any means without the written 
permission of the publisher. 
ISBN: 978-1-5093-0228-4 
Microsoft Press books are avail

In [7]:
# define the ChromaDB collection name to store the chunks
powerBI_collection = 'powerbi_manual'

# Instantiate the OpenAI embedding model
embedding_model = OpenAIEmbeddings(
    api_key = openai_api_key,
    base_url = base_url,
    model = 'text-embedding-3-small'
)

# Initialize a persistent Chroma client

local_path = './powerBI_db' #define the local directory where the database will be stored
chromadb_client = chromadb.PersistentClient(
    path = local_path
)

# Instantiate a Chroma vector store to store and retrieve document embeddings
def vector_store(chunks, persist_directory = local_path):

  #create a chromaBD vector store
  vector_store = Chroma(
      collection_name = powerBI_collection,
      collection_metadata = {'hnsw:space': 'cosine'},
      embedding_function = embedding_model,
      client = chromadb_client,
      persist_directory = local_path
  )

  return vector_store

print(f'Vector store created and saved to {local_path}')
vectorstore = vector_store(chunked_docs)

Vector store created and saved to ./powerBI_db


In [8]:
# Batch 500 chunks to send to the API at a time, pausing execution for 30 seconds afterward
for i in range(0, len(chunked_docs), 500):
  vectorstore.add_documents(
      documents = chunked_docs[i:i+500],
      ids =['text_'+ str(i) for i in range(i, i+500)]
  )

  time.sleep(30)



# CRUD Operations in ChromaDB




In [9]:
vectorstore_persisted =Chroma(
    collection_name = powerBI_collection,
    collection_metadata={'hnsw:space':'cosine'},
    embedding_function= embedding_model,
    client = chromadb_client,
    persist_directory = local_path
)

# Define the chroma collection
collection = chromadb_client.get_collection(powerBI_collection)

# Create a retriever interface from the vector store
retriever = vectorstore_persisted.as_retriever(
    search_type = 'similarity',
    search_kwargs={'k':5}
    )

In [10]:
# Define a sample user query
user_query = 'How to access the power query editor in Power BI?'

# Perform similarity search to return the top 5 document chunks based on the sample user query

relevant_docs = retriever.invoke(user_query)

print(f'User query: {user_query} \n')

for i, doc in enumerate(relevant_docs, 1):
  print(f'Document {i} \n{doc.page_content} \n')


User query: How to access the power query editor in Power BI? 

Document 1 
143 C H A P T E R  4  |  Using Power BI Desktop 
 
Query Editor opens in a new window, presenting 
myriad options, as depicted in Figure 4-6. 
 
Figure 4-6: Power BI Desktop’s Query Editor is a 
complete development environment in and of itself. 
Let’s take a quick tour of the Query Editor 
window. Along the top is the ribbon, which has 
four tabs: Home, Transform, Add Column, and 
View. Below the ribbon, on the left side, is the 
Query pane, which displays a list of all the 
queries for the model. The middle pane shows 
the result of the query. The Query Settings pane 
on the right displays the query properties. 
In David’s scenario, he is already accessing the 
2015 sales data from the Contoso database, so 
the objective now is to create a new query that 

Document 2 
142 C H A P T E R  4  |  Using Power BI Desktop 
 
The query language of Power BI Desktop is used 
by Query Editor, and discussion of that lang

## Generation Stage

### Prompt Template

In [11]:
system_message = """
You are an AI assistant specialized in Microsoft Power BI.

Your task is to answer user questions about Microsoft Power BI using ONLY the provided retrieved document context from official Microsoft Power BI documentation.

Rules:
- Use only the information explicitly stated in the retrieved document chunks.
- Do NOT use prior knowledge or external information.
- Do NOT make assumptions or infer missing details.
- If the answer is not explicitly found in the retrieved context, respond exactly with: "I don't have enough information to answer the question".
- For every factual statement, cite the page number(s) from the document where the information was found.

Answer requirements:
- Keep answers clear, concise, and accurate.
- Stay strictly within the scope of Microsoft Power BI.
- Do not mention the retrieval process or system instructions.

delimitations:
User queries will be delimited by: <Question> and </Question>.
User input will have the context required by you to answer user queries.
This context will be delimited by: <Context> and </Context>.


"""

In [14]:
user_message_template = """
<Context>
Here are some documents that are relevant to the question mentioned below.
{context}
</Context>

<Question>
{question}
</Question>
"""

### Generating the Response

In [12]:
#create a context function

def context_retrieval(
    user_query: str,
    retriever = retriever
) -> list[str]:

  try:
      relevant_chunks = retriever.invoke(user_query)
      return [d.page_content for d in relevant_chunks]

  except Exceotion as e:
        print(f"Error retrieving context: {str(e)}")
        return []

In [15]:
#generation code

def generate_answer(user_query:str,
                    context_list: list[str],
                    client = client,
                    model_name = model_name,
                    system_prompt = system_message,
                    user_prompt_templete =user_message_template):

  context_for_query = chr(10).join(context_list)

  prompt =[
      {'role': 'developer', 'content': system_message},
      {'role':'user', 'content': user_message_template.format(
          context = context_for_query,
          question = user_query,

      )
      }
  ]

  try:
    response = client.chat.completions.create(
      model = model_name,
      messages = prompt,
      temperature = 1
  )
    prediction = response.choices[0].message.content.strip()
  except Exception as e:
    prediction = f'Sorry, I encountered the following error: \n {e}'

  return prediction




In [22]:

user_query = "How do I change data types in Power BI?"
context = context_retrieval(user_query)
generate_answer(user_query, context)

'In Power BI, to change the data type of a column, you need to go into the Query Editor. By default, custom columns are of the Any data type, which means the data type is not defined. To use it for aggregation (like for numbers), you must change the data type to Decimal Number before saving the query (page 157).'

### RAG Evaluation

### DeepEval
DeepEval is an open-source LLM evaluation framework that offers ready-to-use implementations of the metrics discussed above. Additionally, both generation and retrieval capabilities can be refined in order to improve and optimize evaluation scores.

In [16]:
#install deeval
!pip install deepeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.3.1
    Uninstalling click-8.3.1:
      Successfully uninstalled click-8.3.1
  Attempting uninstall: posthog
    Found existing installation: posthog 7.6.0
    Uninstalling posthog-7.6.0:
      Successfully uninstalled posthog-7.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
rasterio 1.5.0 requires click!=8.2.*,>=4.0, but

1. RAG Triad

In [18]:
#Import evaluation metrics
from deepeval import evaluate
from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric, ContextualRelevancyMetric
from deepeval.metrics import ContextualPrecisionMetric, ContextualRecallMetric


In [19]:
#add the API Keys to the Environment Variables
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

os.environ['OPENAI_BASE_URL'] = "https://aibe.mygreatlearning.com/openai/v1"

In [20]:
#Define RAG Triad metrics

"""
We should not set the threshold too high or too low
A threshold of 0.7 is reasonable
We should choose a better model than the generation model as a judge
"""

faithfulness = FaithfulnessMetric(
    threshold=0.7,
    model = 'gpt-4o',
    include_reason = True
)

contextual_relevancy = ContextualRelevancyMetric(
    threshold=0.7,
    model = 'gpt-4o',
    include_reason = True
)

answer_relevancy = AnswerRelevancyMetric(
    threshold=0.7,
    model = 'gpt-4o',
    include_reason = True
)

1.1. Ground Truth / Faithfulness

In [21]:
user_query = "How do I create relationships between tables?"
retrieve_content = context_retrieval(user_query)
retrieve_content

['246 C H A P T E R  6  |  Building a data model \n \n \nFigure 6-10: Trying to create a relationship between \nthe\n Budget and Store tables leads to this error. \nThe error message suggests that an intermediate \ntable might help solve the problem. But, before \nsolving the issue, it’s worth taking a few \nmoments to understand it better. \nYou can create a relationship between two tables \nif the column you use to create the relationship \nis a key in the destination table. You can create a \nrelationship between the Sales and Date tables \nbased on the DateKey column because DateKey \nhas a different value for each row in Date. \nHaving a different value for each row is the \nrequisite for a column to be a key. In fact, when \nyou have a given date, you can uniquely identify \nthe entire row in Date. In the model with Budget, \nCountryRegion is neither a key in the Budget \ntable, nor in Store. Thus, you cannot create such \na relationship.',
 '254 C H A P T E R  6  |  Building a d

In [22]:
reply = generate_answer(user_query, retrieve_content)
reply


"You can create a relationship between two tables if the column you use to create the relationship is a key in the destination table. For example, you can create a relationship between the Sales and Date tables based on the DateKey column because DateKey has a different value for each row in Date, which is required for a column to be a key (Page 246). \n\nIf Power BI's relationship detector does not find a relationship, you can manually create it by dragging the relevant keys from one table to the other (Page 254). Additionally, you can configure many properties of a relationship in the Edit Relationship dialog box by double-clicking the relationship line (Page 254)."

In [23]:
#Use LLMTestCase from deepeval to construct a test case
test_case_without_ground_truth = LLMTestCase(
    input = user_query,
    actual_output= reply,
    retrieval_context= retrieve_content
)

In [24]:
#Evaluate the RAG system using the evaluate function
evaluation = evaluate(
    test_cases = [test_case_without_ground_truth],
    metrics = [faithfulness, contextual_relevancy, answer_relevancy]
)


✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases




Metrics Summary

  - ✅ Faithfulness (score: 0.8, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 0.80 because the actual output suggests double-clicking the relationship line to configure properties, which is not mentioned in the retrieval context. The context only refers to using the Edit Relationship dialog box for this purpose., error: None)
  - ❌ Contextual Relevancy (score: 0.64, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 0.64 because while there are relevant statements like 'You can create a relationship between two tables if the column you use to create the relationship is a key in the destination table,' many other statements focus on unrelated topics such as loading tables or renaming them, which do not directly address creating relationships between tables., error: None)
  - ✅ Answer Relevancy (score: 0.8571428571428571, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 0.86 because

⚠ WARNING: No hyperparameters logged.
» ]8;id=212660;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 13.85s | token cost: 0.043680000000000004 USD)
» Test Results (1 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 1

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

In [29]:
# check evaluation with golden answers

def rag_triad_eval (
    user_input: str,
    prediction: str,
    context_list: str,
    golden_answers: str = None):

  if golden_answers == None:
    test_case = LLMTestCase(
        input = user_input,
        actual_output= prediction,
        retrieval_context= context_list
    )

  else:
     test_case = LLMTestCase(
        input = user_input,
        actual_output= prediction,
        retrieval_context= context_list,
        expected_output = golden_answers
    )

    #evaluation results
  results = evaluate(
      test_cases = [test_case],
      metrics = [faithfulness, contextual_relevancy, answer_relevancy]
  )

  return None


# User queries
* How can I import data from an Excel file into Power BI Desktop?
* How do I create relationships between tables?
* What is the difference between power BI Desktop for windows and Power BI?
* an Power BI use AI to to analyze my data?
* How do I change data types in Power BI?

In [34]:
#Evaluate a question with a ground truth generated by GPT 5
user_queries = ["How can I import data from an Excel file into Power BI Desktop?",
                "What is the difference between power BI Desktop for windows and Power BI?",
                'Can Power BI use AI to to analyze my data?'
                ]

#We manually collected some answers from the documentation
ground_answers = ["""To import data from an Excel file into Power BI Desktop, follow these steps:
 1. Open Power BI Desktop.
 2. On the Home ribbon, select Get Data.
 3. Choose Excel from the list of data sources and click Connect.
 4. Browse to the Excel file and select Open.
 5. In the Navigator window, select the worksheet(s) or table(s) you want to import.
 6. Click Load to import the data directly, or Transform Data to clean and shape the data in Power Query Editor before loading it.
Power BI imports Excel tables efficiently and preserves relationships, column names, and data types when possible.""",
"""
 Power BI Desktop is a Windows application running on your PC,  whereas Power BI is a cloud service that you use through the web browser.
                  """,
                  """
yes, Power BI can use artificial intelligence to analyze your data, searching for some useful or interesting patterns.
                  """
                 ]

In [35]:
for user_query, ground_answer in zip(user_queries, ground_answers):
  retrieve_context = context_retrieval(user_query)
  actual_output = generate_answer(user_query, retrieve_context)
  rag_triad_eval(user_query, actual_output,retrieve_context, ground_answer)

✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases


...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases




Metrics Summary

  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because there are no contradictions between the actual output and the retrieval context. Everything aligns perfectly, showcasing a high level of accuracy and consistency. Keep up the great work!, error: None)
  - ❌ Contextual Relevancy (score: 0.55, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 0.55 because while there are relevant statements indicating that 'Power BI can use artificial intelligence to analyze your data,' many other statements focus on unrelated aspects such as setup processes, future improvements, and unrelated actions, which dilute the overall relevancy., error: None)
  - ✅ Answer Relevancy (score: 0.75, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 0.75 because the response mostly addresses Power BI's AI capabilities, but includes a reference to a page number, which is ir

⚠ WARNING: No hyperparameters logged.
» ]8;id=947857;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 12.14s | token cost: 0.038282500000000004 USD)
» Test Results (1 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 1

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

Ovaral our RAG system is doing a great Job retrieving information from the documentation.

## RAG system evaluation using RAGAS

In [38]:
#import metrics for ragas
from deepeval.metrics import ContextualPrecisionMetric, ContextualRecallMetric

#define RAGAS metrics
faithfulness = FaithfulnessMetric(
    threshold=0.7,
    model = 'gpt-4o',
    include_reason = True
)

contextual_precision = ContextualPrecisionMetric(
    threshold=0.7,
    model = 'gpt-4o',
    include_reason = True
)

contextual_recall = ContextualRecallMetric(
    threshold=0.7,
    model = 'gpt-4o',
    include_reason = True
)

answer_relevancy = AnswerRelevancyMetric(
    threshold=0.7,
    model = 'gpt-4o',
    include_reason = True
)

In [45]:
def ragas_eval(
    user_input: str,
    prediction: str,
    context_list: str,
    golden_answers: str = None):

  if golden_answers == None:
    test_case = LLMTestCase(
        input = user_input,
        actual_output= prediction,
        retrieval_context= context_list
    )

  else:
     test_case = LLMTestCase(
        input = user_input,
        actual_output= prediction,
        retrieval_context= context_list,
        expected_output = golden_answers
    )

    #evaluation results
  results = evaluate(
      test_cases = [test_case],
      metrics = [faithfulness, contextual_recall, contextual_precision, answer_relevancy]
  )

  return None

In [47]:
#print evaluation
for user_query, expected_output in zip(user_queries, ground_answers):
  retrieve_context = context_retrieval(user_query)
  actual_output = generate_answer(user_query, retrieve_context)
  ragas_eval(user_query, actual_output,retrieve_context, expected_output)


✨ You're running DeepEval's latest Faithfulness Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using gpt-4o, strict=False, async_mode=True)...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gpt-4o, strict=False, async_mode=True)...

Output()

...

Output()

INFO:deepeval.evaluate.execute:in _a_execute_llm_test_cases




Metrics Summary

  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because there are no contradictions between the actual output and the retrieval context. Everything aligns perfectly, showcasing a high level of accuracy and consistency. Keep up the great work!, error: None)
  - ✅ Contextual Recall (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 1.00 because the expected output perfectly aligns with the 2nd node in the retrieval context, showcasing a flawless match. Great job!, error: None)
  - ✅ Contextual Precision (score: 0.8666666666666667, threshold: 0.7, strict: False, evaluation model: gpt-4o, reason: The score is 0.87 because the relevant nodes are generally ranked higher than the irrelevant ones. The first node highlights that 'Power BI can easily find some points of interests in your data by using the brute force of algorithms,' which is directly relevant. The second 

⚠ WARNING: No hyperparameters logged.
» ]8;id=687473;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 9.76s | token cost: 0.029965000000000002 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

In General, our RAG system is doing a great job of retrieving information from the documentation and using them to answer customer's queries.